In [13]:
from dotenv import load_dotenv
load_dotenv('.env.local')
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


: 

In [14]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.delete_index("rag")
pc.create_index(name="rag", dimension=768, metric="cosine", spec=ServerlessSpec("aws", "us-east-1"))

In [15]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Sarah Johnson',
  'subject': 'Physics',
  'grade': 'A',
  'star': 5,
  'review': "Dr. Johnson's lectures are incredibly engaging. She explains complex concepts with clarity and enthusiasm."},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'grade': 'B+',
  'star': 4,
  'review': 'Prof. Chen is knowledgeable, but his assignments can be quite challenging. Office hours are very helpful.'},
 {'professor': 'Dr. Emily Rodriguez',
  'subject': 'Biology',
  'grade': 'A-',
  'star': 5,
  'review': "Dr. Rodriguez's passion for biology is contagious. Her labs are well-organized and interesting."},
 {'professor': 'Prof. David Thompson',
  'subject': 'History',
  'grade': 'B',
  'star': 3,
  'review': 'Prof. Thompson knows his subject well, but his lectures can be dry at times. Exams are fair.'},
 {'professor': 'Dr. Lisa Patel',
  'subject': 'Psychology',
  'grade': 'A',
  'star': 5,
  'review': 'Dr. Patel creates a supportive learning environment. Her re

In [16]:
import torch
from transformers import BertTokenizer, BertModel

#  Load pre-trained model tokenizer (vocabulary) and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    # Tokenize input
    input = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Get the hidden states
    with torch.no_grad():
        outputs = model(**input)
    # use the embeddings of the first token (CLS) as the sentence embedding
    embedding = outputs.last_hidden_state[:,0,:].squeeze().tolist()
    return embedding

processed_data = []
for review in data['reviews']:
    embedding = get_bert_embedding(review['review'])
    processed_data.append({
        "values": embedding,
        "id": review['professor'],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "star": review["star"],
        }
    })

In [17]:
processed_data[0]

{'values': [-0.025057770311832428,
  0.00429186224937439,
  -0.34080490469932556,
  -0.0005305297672748566,
  0.3961600661277771,
  -0.3898257911205292,
  0.291171133518219,
  0.4654960632324219,
  0.18764688074588776,
  -0.6066188812255859,
  0.31142526865005493,
  0.12639230489730835,
  0.5738189220428467,
  0.6511151194572449,
  0.14667251706123352,
  0.4333610534667969,
  0.3051088750362396,
  0.7262204885482788,
  0.1991943120956421,
  -0.22513051331043243,
  0.0491785928606987,
  -0.774391770362854,
  0.29340916872024536,
  0.1553390622138977,
  0.4419828951358795,
  -0.17630548775196075,
  -0.34050166606903076,
  0.1448935866355896,
  0.17412185668945312,
  -0.42086753249168396,
  -0.3650224208831787,
  0.15951409935951233,
  -0.2914607524871826,
  -0.7533026933670044,
  0.17529956996440887,
  0.027933254837989807,
  0.1973191648721695,
  0.08553847670555115,
  -0.09635022282600403,
  -0.05039946362376213,
  -0.4839772880077362,
  -0.06526437401771545,
  0.24637067317962646,
  -

In [18]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}